In [1]:
import os
from dotenv import load_dotenv

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [19]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain.schema import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#### INDEXING ####

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is the process of breaking down a complex task into smaller, more manageable steps. Techniques like Chain of Thought (CoT) and Tree of Thoughts (ToT) are used to enhance model performance by instructing the model to think step by step and explore multiple reasoning possibilities. This can be achieved through simple prompting, task-specific instructions, or human inputs.'

In [21]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

8

Text embedding models

In [22]:
from langchain_openai import OpenAIEmbeddings
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result)

1536

cosine similarity

In [23]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.8807044730847644


Loading Document

In [24]:
import fitz  # PyMuPDF
from langchain.document_loaders import TextLoader

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

In [26]:
contract_text_raptor = extract_text_from_pdf('../data/Evaluation Sets/Raptor Contract.pdf')
qa_text_raptor = extract_text_from_pdf('../data/Evaluation Sets/Raptor Q&A.pdf')
contract_text_robinson = extract_text_from_pdf('../data/Evaluation Sets/Robinson Advisory.pdf')
qa_text_robinson = extract_text_from_pdf('../data/Evaluation Sets/Robinson Q&A.pdf')

In [27]:
# Chunking
def chunk_text(text, chunk_size=500, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = ' '.join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

In [28]:
contract_chunks_raptor = chunk_text(contract_text_raptor, chunk_size=300, overlap=50)

In [30]:
documents_raptor = [Document(page_content=chunk) for chunk in contract_chunks_raptor]

Vectorstores

In [36]:
vectorstore_raptor = FAISS.from_documents(documents_raptor, embeddings)

### Retrieval

In [43]:
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain

In [48]:
qa_pipeline_raptor = RetrievalQA.from_llm(llm=llm, retriever=vectorstore_raptor.as_retriever())

### Generation

In [53]:
qa_pairs_raptor = 

In [54]:
# Run evaluation
accuracy_raptor = evaluate_model(qa_pipeline_raptor, qa_pairs_raptor)
print(f"Raptor Contract Accuracy: {accuracy_raptor * 100:.2f}%")

Question: What is the term of the contract?
Expected: The term of the contract is 12 months.
Generated: The provided context does not specify the term or duration of the contract. If you need information about the term of the contract, you would need to refer to the specific section of the agreement that outlines the duration or term, which is not included in the excerpts provided.
---
Question: Are there any conditions to the closing?
Expected: No, as the signing and closing are simultaneous.
Generated: The provided context does not explicitly list the conditions to the closing. It mentions various aspects of the agreement, such as the purchase and sale of shares, purchase price adjustments, and the treatment of employees and options, but it does not detail specific conditions that must be met for the closing to occur. If you need information on specific conditions to the closing, you may need to refer to other sections of the agreement not included in the provided context.
---
Raptor

In [49]:
# Evaluation 
def evaluate_model(qa_pipeline, qa_pairs):
    correct = 0
    total = len(qa_pairs)
    
    for qa in qa_pairs:
        question = qa['question']
        expected_answer = qa['answer']
        
        # Get the generated answer from the RAG pipeline
        generated_answer = qa_pipeline.run(question)
        
        # Compare the generated answer to the expected answer
        if generated_answer.strip().lower() == expected_answer.strip().lower():
            correct += 1
        else:
            print(f"Question: {question}")
            print(f"Expected: {expected_answer}")
            print(f"Generated: {generated_answer}")
            print("---")
    
    accuracy = correct / total
    return accuracy